Importiamo Pyomo e GLPK

In [1]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

Retrieving notices: ...working... done
Solving environment: done

## Package Plan ##

  environment location: /Users/giovanni/opt/anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-23.1.0               |   py39h6e9494a_0         908 KB  conda-forge
    glpk-5.0                   |       h3cb5acd_0         1.0 MB  conda-forge
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.9-2_cp39 

The following packages will be UPDATED:

  glpk                      pkgs/main::glpk-4.65-h05f8d05_3 --> conda-forge::glpk-5.0-h3cb5acd_0 

The following packages will be SUPERSEDED by a higher-priori

Vogliamo modellare il problema del Bin Packing dato un insieme di oggetti $ITEMS$, un insieme di bin identici $BINS$ e un vettore di pesi $P$

$~$

Modello ILP:

$\min \sum_{j \in BINS} y_j $

$\sum_{j \in BINS} x_{ij}=1 ~~~~~i \in ITEMS$

$\sum_{i \in ITEMS} w_i x_{ij} \le y_jW ~~~~~j \in BINS$

$x_{ij} \in \{0,1\} ~~~~~~ i \in ITEMS ~~, j \in BINS$

$y_j \in \{0,1\} ~~~~~~ j \in BINS$


In [39]:
ITEMS = ['calzini','pantaloni','fiammiferi','spazzolino','mattone']

BINS = ['bin1','bin2','bin3','bin4','bin5']
W = 10
p=[7,4,1,4,4]

In [ ]:
#PESI = {'calzini':7,'pantaloni':4,'fiammiferi':1,'spazzolino':4,'mattone':4} #utilizziamo le liste per assegnare in peso agli oggetti
PESI = dict(zip(ITEMS,p))
print (PESI)

Importiamo la libreria PYOMO

In [40]:
from pyomo.environ import * #qui andrebbe a seguire un nome che diventi quella che usiamo nel codice, best practice

In [41]:
BPP = ConcreteModel() #chiamo il modello mix

Definiamo delle variabili non negative

In [42]:
BPP.x = Var(ITEMS, BINS, within=Binary)
BPP.y = Var(BINS, within=Binary)

Definiamo la funzione obiettivo ma per farlo definiamo una funzione che prende come argomento il modello(chiamato genericamente mod, parametro informale) che poi passiamo alla funzione come rule

In [43]:
def obj_rule(mod):
  return sum(mod.y[j] for j in BINS)
BPP.obj = Objective(rule=obj_rule, sense = minimize) #la funzione obiettivo si aspetta o una espressione o una regola che restituisca un valore ma non prende in input un valore

In [44]:
def packed_rule(mod, i): #gli passo i come elemento dell'insieme ITEMS
  return sum(mod.x[i,j] for j in BINS) == 1
BPP.pack_const = Constraint(ITEMS, rule=packed_rule) #qui pyomo prende ogni elemento dell'insieme ITEMS, chiama la funzione

Allo stesso modo definiamo i vincoli. Innanzitutto definiamo una funzione che prende come argomento il modello (mod) e un parametro m.

Attenzione: abbiamo un vincolo per ogni materiale, cioe per ogni elemento $m$ della lista $M$

In [45]:
def weight_rule(mod, j):
  return sum(PESI[i] * mod.x[i,j] for i in ITEMS) <= W * mod.y[j]
BPP.capacity_const = Constraint(BINS, rule=weight_rule) #qui pyomo prende ogni elemento dell'insieme ITEMS, chiama la funzione

In [46]:
SolverFactory('glpk').solve(BPP, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmp49jwmc8u.glpk.raw
 --wglp /var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpkng6btq3.glpk.glp
 --cpxlp /var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpelu6u1oy.pyomo.lp
Reading problem data from '/var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpelu6u1oy.pyomo.lp'...
/var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpelu6u1oy.pyomo.lp:133: warning: lower bound of variable 'x1' redefined
/var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpelu6u1oy.pyomo.lp:133: warning: upper bound of variable 'x1' redefined
11 rows, 31 columns, 56 non-zeros
30 integer variables, all of which are binary
163 lines were read
Writing problem data to '/var/folders/b2/kqnx_r5555gfw4sz0bdz9b440000gp/T/tmpkng6btq3.glpk.glp'...
118 lines were written
GLPK Integer Optimizer, v4.65
11 rows, 31 columns, 56 non-zeros
30 integer variables, all of which are binary
Preproc

{'Problem': [{'Name': 'unknown', 'Lower bound': 3.0, 'Upper bound': 3.0, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 31, 'Number of nonzeros': 56, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '31', 'Number of created subproblems': '31'}}, 'Error rc': 0, 'Time': 0.04834794998168945}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [47]:
BPP.pprint()

6 Set Declarations
    capacity_const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'bin1', 'bin2', 'bin3', 'bin4', 'bin5'}
    pack_const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'calzini', 'pantaloni', 'fiammiferi', 'spazzolino', 'mattone'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   25 : {('calzini', 'bin1'), ('calzini', 'bin2'), ('calzini', 'bin3'), ('calzini', 'bin4'), ('calzini', 'bin5'), ('pantaloni', 'bin1'), ('pantaloni', 'bin2'), ('pantaloni', 'bin3'), ('pantaloni', 'bin4'), ('pantaloni', 'bin5'), ('fiammiferi', 'bin1'), ('fiammiferi', 'bin2'), ('fiammiferi', 'bin3'), ('fiammiferi', 'bin4'), ('fiammiferi', 'bin5'), ('spazzolino', 'bin1'), ('spazzolino', 'bin2'), ('spazzolino', 'bin3'), ('spaz